# 前準備
駅データから各種データをダウンロードしておく。（会員登録が必要）
https://ekidata.jp/dl/

- 事業者データ
- 路線データ
- 駅データ
- 接続駅データ

各データの仕様はこちら。
https://ekidata.jp/doc/

# 事業者データ
今回使用するのは 20230105 時点の version のデータ。
仕様はこちら。
https://ekidata.jp/doc/company.php

In [2]:
import pandas as pd

In [5]:
df_company = pd.read_csv('../files/ekidata/company20230105.csv')

In [6]:
df_company.head()

,company_cd,rr_cd,company_name,company_name_k,company_name_h,company_name_r,company_url,company_type,e_status,e_sort
0,1,11,JR北海道,ジェイアールホッカイドウ,北海道旅客鉄道株式会社,JR北海道,http://www.jrhokkaido.co.jp/,1,0,1
1,2,11,JR東日本,ジェイアールヒガシニホン,東日本旅客鉄道株式会社,JR東日本,http://www.jreast.co.jp/,1,0,2
2,3,11,JR東海,ジェイアールトウカイ,東海旅客鉄道株式会社,JR東海,http://jr-central.co.jp/,1,0,3
3,4,11,JR西日本,ジェイアールニシニホン,西日本旅客鉄道株式会社,JR西日本,http://www.westjr.co.jp/,1,0,4
4,5,11,JR四国,ジェイアールシコク,四国旅客鉄道株式会社,JR四国,http://www.jr-shikoku.co.jp/,1,0,5


In [8]:
df_company.shape

(171, 10)

意外と行数が多い。鉄道関係は全然詳しくないのだけど、想像以上に事業者が多いみたい。

## 欠損値の確認

In [9]:
df_company.isna().sum().reset_index()

,index,0
0,company_cd,0
1,rr_cd,0
2,company_name,0
3,company_name_k,0
4,company_name_h,0
5,company_name_r,0
6,company_url,7
7,company_type,0
8,e_status,0
9,e_sort,0


`compamy_url` が欠損しているものが7行。他のカラムでは欠損は無い。

In [18]:
df_company[df_company.company_url.isnull()]

,company_cd,rr_cd,company_name,company_name_k,company_name_h,company_name_r,company_url,company_type,e_status,e_sort
24,103,99,北海道ちほく高原鉄道,ホッカイドウチホクコウゲンテツドウ,北海道ちほく高原鉄道株式会社,北海道ちほく高原鉄道,NaN,0,2,103
35,113,99,くりはら田園鉄道,クリハラデンエンテツドウ,くりはら田園鉄道株式会社,栗原電鉄,NaN,0,2,113
83,161,99,神岡鉄道,カミオカテツドウ,神岡鉄道株式会社,神鉄,NaN,0,2,161
105,180,99,桃花台新交通,トウカダイシンコウツウ,桃花台新交通株式会社,桃花台新交通,NaN,0,2,178
115,247,99,四日市あすなろう鉄道,ヨッカイチアスナロウテツドウ,四日市あすなろう鉄道株式会社,あすてつ,NaN,0,0,187
135,208,99,三木鉄道,ミキテツドウ,三木鉄道株式会社,三木鉄道,NaN,0,2,206
147,220,99,スカイレールサービス,スカイレールサービス,スカイレールサービス株式会社,スカイレールサービス,NaN,0,0,218


`compamy_url` が欠損している行を確認。`e_status` が`2`（廃止）の事業者を除くと以下2つの事業者。
- 四日市あすなろう鉄道株式会社
  - https://yar.co.jp/ だと思うが、単純な漏れかもしれない。
- スカイレールサービス株式会社
  - 探してもそれらしいものは発見できなかった。存在しないのであれば NaN なのは納得。
  - wikipedia によると `2023年度末での運行終了` とのことなので、いずれ `e_status` が `2` に更新されるのだろう。

## 鉄道コード `rr_cd`

In [23]:
df_company.groupby('rr_cd').size().reset_index()

,rr_cd,0
0,11,6
1,21,1
2,22,1
3,23,1
4,24,1
5,25,1
6,26,1
7,27,1
8,28,1
9,29,1


`11` と `99` 以外はすべて1行のみ。`11` も6行のみで、残りすべてが `99` だが、仕様を見ても鉄道コードについての説明がほぼ無いため、何を表しているのかはよくわからない。

とりあえず `11` を見てみよう。

In [25]:
df_company[df_company.rr_cd == 11]

,company_cd,rr_cd,company_name,company_name_k,company_name_h,company_name_r,company_url,company_type,e_status,e_sort
0,1,11,JR北海道,ジェイアールホッカイドウ,北海道旅客鉄道株式会社,JR北海道,http://www.jrhokkaido.co.jp/,1,0,1
1,2,11,JR東日本,ジェイアールヒガシニホン,東日本旅客鉄道株式会社,JR東日本,http://www.jreast.co.jp/,1,0,2
2,3,11,JR東海,ジェイアールトウカイ,東海旅客鉄道株式会社,JR東海,http://jr-central.co.jp/,1,0,3
3,4,11,JR西日本,ジェイアールニシニホン,西日本旅客鉄道株式会社,JR西日本,http://www.westjr.co.jp/,1,0,4
4,5,11,JR四国,ジェイアールシコク,四国旅客鉄道株式会社,JR四国,http://www.jr-shikoku.co.jp/,1,0,5
5,6,11,JR九州,ジェイアールキュウシュウ,九州旅客鉄道株式会社,JR九州,http://www.jrkyushu.co.jp/,1,0,6


`11` は JR ということがわかった。

`99` は数が多いので後回しにして、`11` と `99` 以外を見てみよう。

In [28]:
df_company[~df_company.rr_cd.isin([11, 99])]

,company_cd,rr_cd,company_name,company_name_k,company_name_h,company_name_r,company_url,company_type,e_status,e_sort
6,11,21,東武鉄道,トウブテツドウ,東武鉄道株式会社,東武,http://www.tobu.co.jp/,2,0,11
7,12,22,西武鉄道,セイブテツドウ,西武鉄道株式会社,西武,http://www.seibu-group.co.jp/railways/,2,0,12
8,13,23,京成電鉄,ケイセイデンテツ,京成電鉄株式会社,京成,http://www.keisei.co.jp/,2,0,13
9,14,24,京王電鉄,ケイオウデンテツ,京王電鉄株式会社,京王,http://www.keio.co.jp/,2,0,14
10,15,25,小田急電鉄,オダキュウデンテツ,小田急電鉄株式会社,小田急,http://www.odakyu.jp/,2,0,15
11,16,26,東急電鉄,トウキュウデンテツ,東京急行電鉄株式会社,東急,http://www.tokyu.co.jp/,2,0,16
12,17,27,京急電鉄,ケイキュウデンテツ,京浜急行電鉄株式会社,京急,http://www.keikyu.co.jp/,2,0,17
13,18,28,東京メトロ,トウキョウメトロ,東京地下鉄株式会社,東京メトロ,http://www.tokyometro.jp/,2,0,18
14,19,29,相模鉄道,サガミテツドウ,相模鉄道株式会社,相鉄,http://www.sotetsu.co.jp/,2,0,19
15,20,30,名古屋鉄道,ナゴヤテツドウ,名古屋鉄道株式会社,名鉄,http://www.meitetsu.co.jp/,2,0,20


`21`~`36` は大手の私鉄が該当していそう。`99`は地方の鉄道会社など JR と大手私鉄以外のすべてが含まれていると思われる。

## 事業者区分 `company_type`

仕様によると `0`:その他　`1`:JR `2`:大手私鉄　`3`:準大手私鉄 という定義。

In [29]:
df_company.groupby('company_type').size().reset_index()

,company_type,0
0,0,143
1,1,6
2,2,17
3,3,5


JRが `6` というのは `rr_cd` の集計とも一致する。大手私鉄は `17` なのですが、`rr_cd` の集計では `16` だった。差異が気になるので調べて見る。

In [30]:
df_company[df_company.company_type == 2]

,company_cd,rr_cd,company_name,company_name_k,company_name_h,company_name_r,company_url,company_type,e_status,e_sort
6,11,21,東武鉄道,トウブテツドウ,東武鉄道株式会社,東武,http://www.tobu.co.jp/,2,0,11
7,12,22,西武鉄道,セイブテツドウ,西武鉄道株式会社,西武,http://www.seibu-group.co.jp/railways/,2,0,12
8,13,23,京成電鉄,ケイセイデンテツ,京成電鉄株式会社,京成,http://www.keisei.co.jp/,2,0,13
9,14,24,京王電鉄,ケイオウデンテツ,京王電鉄株式会社,京王,http://www.keio.co.jp/,2,0,14
10,15,25,小田急電鉄,オダキュウデンテツ,小田急電鉄株式会社,小田急,http://www.odakyu.jp/,2,0,15
11,16,26,東急電鉄,トウキュウデンテツ,東京急行電鉄株式会社,東急,http://www.tokyu.co.jp/,2,0,16
12,17,27,京急電鉄,ケイキュウデンテツ,京浜急行電鉄株式会社,京急,http://www.keikyu.co.jp/,2,0,17
13,18,28,東京メトロ,トウキョウメトロ,東京地下鉄株式会社,東京メトロ,http://www.tokyometro.jp/,2,0,18
14,19,29,相模鉄道,サガミテツドウ,相模鉄道株式会社,相鉄,http://www.sotetsu.co.jp/,2,0,19
15,20,30,名古屋鉄道,ナゴヤテツドウ,名古屋鉄道株式会社,名鉄,http://www.meitetsu.co.jp/,2,0,20


`Osaka Metro` が大手私鉄と判定されているようだ。`rr_cd` は定義がわからず、内容から `11` `99` 以外は大手私鉄と予想したのだが、これでさらに定義の謎が深まってしまったｗ

# 状態 `e_status`
仕様によると `0`:運用中　`1`:運用前　`2`:廃止 という定義。

In [31]:
df_company.groupby('e_status').size().reset_index()

,e_status,0
0,0,163
1,2,8


`20230105` 時点の version では `1`:運用前 という事業者は存在していないようだ。

`2`:廃止 の事業者を見てみよう。

In [32]:
df_company[df_company.e_status == 2]

,company_cd,rr_cd,company_name,company_name_k,company_name_h,company_name_r,company_url,company_type,e_status,e_sort
24,103,99,北海道ちほく高原鉄道,ホッカイドウチホクコウゲンテツドウ,北海道ちほく高原鉄道株式会社,北海道ちほく高原鉄道,NaN,0,2,103
28,106,99,十和田観光電鉄,トワダカンコウデンデツ,十和田観光電鉄株式会社,十和田観光電鉄,http://www.toutetsu.co.jp/,0,2,106
35,113,99,くりはら田園鉄道,クリハラデンエンテツドウ,くりはら田園鉄道株式会社,栗原電鉄,NaN,0,2,113
57,135,99,鹿島鉄道,カシマテツドウ,鹿島鉄道株式会社,鹿鉄,http://www.kantetsu.co.jp/katetsu/,0,2,135
83,161,99,神岡鉄道,カミオカテツドウ,神岡鉄道株式会社,神鉄,NaN,0,2,161
105,180,99,桃花台新交通,トウカダイシンコウツウ,桃花台新交通株式会社,桃花台新交通,NaN,0,2,178
132,205,99,北神急行電鉄,ホクシンキュウコウデンテツ,北神急行電鉄株式会社,北神急行,http://www.hokushinkyuko.co.jp/,0,2,203
135,208,99,三木鉄道,ミキテツドウ,三木鉄道株式会社,三木鉄道,NaN,0,2,206


# 路線データ

In [3]:
df_line = pd.read_csv('./ekidata/line20230320free.csv')

In [4]:
df_line.head()

,line_cd,company_cd,line_name,line_name_k,line_name_h,line_color_c,line_color_t,line_type,lon,lat,zoom,e_status,e_sort
0,1001,3,中央新幹線,チュウオウシンカンセン,中央新幹線,NaN,NaN,NaN,137.493896,35.411438,8,1,1001
1,1002,3,東海道新幹線,トウカイドウシンカンセン,東海道新幹線,NaN,NaN,NaN,137.721489,35.144122,7,0,1002
2,1003,4,山陽新幹線,サンヨウシンカンセン,山陽新幹線,NaN,NaN,NaN,133.147896,34.419338,7,0,1003
3,1004,2,東北新幹線,トウホクシンカンセン,東北新幹線,NaN,NaN,NaN,140.763192,38.274267,7,0,1004
4,1005,2,上越新幹線,ジョウエツシンカンセン,上越新幹線,NaN,NaN,NaN,139.121488,36.798565,8,0,1005


In [6]:
df_line.shape

(620, 13)

カラム数は 13、行数は 620。

In [7]:
df_line.columns

Index(['line_cd', 'company_cd', 'line_name', 'line_name_k', 'line_name_h',
       'line_color_c', 'line_color_t', 'line_type', 'lon', 'lat', 'zoom',
       'e_status', 'e_sort'],
      dtype='object')

In [15]:
df_line.isna().sum().reset_index()

,index,0
0,line_cd,0
1,company_cd,0
2,line_name,0
3,line_name_k,0
4,line_name_h,0
5,line_color_c,572
6,line_color_t,572
7,line_type,620
8,lon,0
9,lat,0


`line_color_c` `line_color_t` `line_type` は有料会員のみのデータになるらしく、今回使用する無料版データでは欠損している。